In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df_obs_daily = pd.read_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2_chirps.csv")
df_obs_daily = df_obs_daily.set_index('Unnamed: 0')

df_obs_monthly = pd.read_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_monthly_v2_chirps.csv")
df_obs_monthly = df_obs_monthly.set_index('Unnamed: 0')

df_sat_daily = pd.read_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_chirps_data_daily_v2_corrincl.csv")
df_sat_daily = df_sat_daily.set_index('Unnamed: 0.1')
df_sat_daily = df_sat_daily.drop(columns=['Unnamed: 0'])

df_sat_monthly = pd.read_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_chirps_data_monthly_v2_corrincl.csv")
df_sat_monthly = df_sat_monthly.set_index('Unnamed: 0.1')

df_obs_metadata = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')
df_obs_metadata = df_obs_metadata.set_index('STNID')

In [ ]:
index = pd.Index(['Lat', 'Lon', 'RMSE', 'MAE', 'Bias', 'PearsonR', 'SpearmanR', 'R2', 'Probability of Detection', 'Probability of False Detection', 'False Alarm Rate', 'Frequency Bias'], name="rows")
column = df_obs_daily.columns

daily_metrics_df = pd.DataFrame(data=np.nan, index=index, columns=column)
monthly_metrics_df = pd.DataFrame(data=np.nan, index=index, columns=column)

for station, station_id in enumerate(df_obs_daily.columns):
  print(station)
  if station == 7 or station == 102:
    continue
  
  obs_daily = df_obs_daily[station_id]  
  sat_daily = df_sat_daily[station_id]

  obs_daily = obs_daily.to_numpy(copy=True)
  sat_daily = sat_daily.to_numpy(copy=True)

  obs_daily = obs_daily[~np.isnan(obs_daily)]
  sat_daily = sat_daily[~np.isnan(sat_daily)]

  n_of_sample = len(obs_daily)

  rmse = mean_squared_error(obs_daily, sat_daily, squared=False)
  mae = mean_absolute_error(obs_daily, sat_daily)
  pearsonr = stats.pearsonr(obs_daily, sat_daily)
  spearmanr = stats.spearmanr(obs_daily, sat_daily)
  rsq = r2_score(obs_daily, sat_daily)

  # correlation_matrix = np.corrcoef(obs_daily, sat_daily)
  # correlation_xy = correlation_matrix[0,1]
  # rsq = correlation_xy**2

  bias = sat_daily - obs_daily
  bias = bias.mean()

  obs_daily_bool = obs_daily.copy()
  sat_daily_bool = sat_daily.copy()

  obs_daily_bool[obs_daily_bool < 1] = 0
  obs_daily_bool[obs_daily_bool >= 1] = 1

  sat_daily_bool[sat_daily_bool < 1] = 0
  sat_daily_bool[sat_daily_bool >= 1] = 1

  category = np.empty([n_of_sample])

  for i in range(n_of_sample):
    if obs_daily_bool[i] == 1 and sat_daily_bool[i] == 1:
      category[i] = 1
    elif obs_daily_bool[i] == 1 and sat_daily_bool[i] == 0:
      category[i] = 2
    elif obs_daily_bool[i] == 0 and sat_daily_bool[i] == 1:
      category[i] = 3
    elif obs_daily_bool[i] == 0 and sat_daily_bool[i] == 0:
      category[i] = 4

  hit = np.count_nonzero(category == 1)
  miss = np.count_nonzero(category == 2)
  false_alarm = np.count_nonzero(category == 3)
  correct_negative = np.count_nonzero(category == 4)

  pod = hit/(hit + miss)
  pofd = miss/(miss+correct_negative)
  far = false_alarm/(hit + false_alarm)
  freq_bias = (hit+false_alarm)/(hit+miss)

  daily_metrics_df[station_id].loc['Lat'] = df_obs_metadata['LATITUDE'].loc[station_id]
  daily_metrics_df[station_id].loc['Lon'] = df_obs_metadata['LONGITUDE'].loc[station_id]
  daily_metrics_df[station_id].loc['RMSE'] = round(rmse,2)
  daily_metrics_df[station_id].loc['MAE'] = round(mae,2)
  daily_metrics_df[station_id].loc['Bias'] = round(bias,2)
  daily_metrics_df[station_id].loc['PearsonR'] = round(pearsonr[0],2)
  daily_metrics_df[station_id].loc['SpearmanR'] = round(spearmanr[0],2)
  daily_metrics_df[station_id].loc['R2'] = round(rsq,2)
  daily_metrics_df[station_id].loc['Probability of Detection'] = round(pod,2)
  daily_metrics_df[station_id].loc['Probability of False Detection'] = round(pofd,2)
  daily_metrics_df[station_id].loc['False Alarm Rate'] = round(far,2)
  daily_metrics_df[station_id].loc['Frequency Bias'] = round(freq_bias,2)
                                  
  print(station_id)
  print("Number of Observation: ", n_of_sample)
  print("RMSE: ", str(round(rmse,1)))
  print("MAE: ", str(round(mae,1)))
  print("Bias: ", str(round(bias,1)))
  print("Rsquare: ", str(round(rsq,1)))
  print("Probability of Detection: ", pod)
  print("Probability of False Detection: ", pofd)
  print("False Alarm Rate: ", far)
  print("Frequency Bias: ", freq_bias)


  # # print(obs_daily)
  # # print(sat_daily)

  # # print(obs_daily_bool)
  # # print(sat_daily_bool)
  print(" ")

  # obs_monthly = df_obs_monthly[station_id]
  # sat_monthly = df_sat_monthly[station_id]

  # obs_monthly = obs_monthly.to_numpy()
  # sat_monthly = sat_monthly.to_numpy()

  # print(station, station_id)

daily_metrics_df = daily_metrics_df.T
daily_metrics_df.to_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/metrics/metrics_chirps_daily.csv')      

In [5]:
index = pd.Index(['Lat', 'Lon', 'ObsN', 'RMSE', 'MAE', 'Bias', 'PearsonR', 'SpearmanR', 'R2', 'POD', 'POFD', 'FAR', 'FB'], name="rows")
column = df_obs_monthly.columns

monthly_metrics_df = pd.DataFrame(data=np.nan, index=index, columns=column)
monthly_metrics_df = pd.DataFrame(data=np.nan, index=index, columns=column)

for station, station_id in enumerate(df_obs_monthly.columns):
  print(station)
  if station == 7 or station == 102:
    continue
  
  obs_monthly = df_obs_monthly[station_id]  
  sat_monthly = df_sat_monthly[station_id]

  obs_monthly = obs_monthly.to_numpy(copy=True)
  sat_monthly = sat_monthly.to_numpy(copy=True)

  obs_monthly = obs_monthly[~np.isnan(obs_monthly)]
  sat_monthly = sat_monthly[~np.isnan(sat_monthly)]

  n_of_sample = len(obs_monthly)

  rmse = mean_squared_error(obs_monthly, sat_monthly, squared=False)
  mae = mean_absolute_error(obs_monthly, sat_monthly)
  pearsonr = stats.pearsonr(obs_monthly, sat_monthly)
  spearmanr = stats.spearmanr(obs_monthly, sat_monthly)
  rsq = r2_score(obs_monthly, sat_monthly)

  # correlation_matrix = np.corrcoef(obs_monthly, sat_monthly)
  # correlation_xy = correlation_matrix[0,1]
  # rsq = correlation_xy**2

  bias = sat_monthly - obs_monthly
  bias = bias.mean()

  obs_monthly_bool = obs_monthly.copy()
  sat_monthly_bool = sat_monthly.copy()

  obs_monthly_bool[obs_monthly_bool < 10] = 0
  obs_monthly_bool[obs_monthly_bool >= 10] = 1

  sat_monthly_bool[sat_monthly_bool < 10] = 0
  sat_monthly_bool[sat_monthly_bool >= 10] = 1

  category = np.empty([n_of_sample])

  for i in range(n_of_sample):
    if obs_monthly_bool[i] == 1 and sat_monthly_bool[i] == 1:
      category[i] = 1
    elif obs_monthly_bool[i] == 1 and sat_monthly_bool[i] == 0:
      category[i] = 2
    elif obs_monthly_bool[i] == 0 and sat_monthly_bool[i] == 1:
      category[i] = 3
    elif obs_monthly_bool[i] == 0 and sat_monthly_bool[i] == 0:
      category[i] = 4

  hit = np.count_nonzero(category == 1)
  miss = np.count_nonzero(category == 2)
  false_alarm = np.count_nonzero(category == 3)
  correct_negative = np.count_nonzero(category == 4)

  if hit + miss == 0:
    pod = np.nan
  else:
    pod = hit/(hit + miss)

  if miss + correct_negative == 0:
    pofd = np.nan
  else:
    pofd = miss/(miss + correct_negative)

  if hit + false_alarm == 0:
    far = np.nan
  else:
    far = false_alarm/(hit + false_alarm)

  if hit + miss == 0:
    freq_bias = np.nan
  else:
    freq_bias = (hit+false_alarm)/(hit+miss)

  monthly_metrics_df[station_id].loc['Lat'] = df_obs_metadata['LATITUDE'].loc[station_id]
  monthly_metrics_df[station_id].loc['Lon'] = df_obs_metadata['LONGITUDE'].loc[station_id]
  monthly_metrics_df[station_id].loc['ObsN'] = n_of_sample
  monthly_metrics_df[station_id].loc['RMSE'] = round(rmse,2)
  monthly_metrics_df[station_id].loc['MAE'] = round(mae,2)
  monthly_metrics_df[station_id].loc['Bias'] = round(bias,2)
  monthly_metrics_df[station_id].loc['PearsonR'] = round(pearsonr[0],2)
  monthly_metrics_df[station_id].loc['SpearmanR'] = round(spearmanr[0],2)
  monthly_metrics_df[station_id].loc['R2'] = round(rsq,2)
  monthly_metrics_df[station_id].loc['POD'] = round(pod,2)
  monthly_metrics_df[station_id].loc['POFD'] = round(pofd,2)
  monthly_metrics_df[station_id].loc['FAR'] = round(far,2)
  monthly_metrics_df[station_id].loc['FB'] = round(freq_bias,2)
                                  
  print(station_id)
  print("Number of Observation: ", n_of_sample)
  print("RMSE: ", str(round(rmse,1)))
  print("MAE: ", str(round(mae,1)))
  print("Bias: ", str(round(bias,1)))
  print("Rsquare: ", str(round(rsq,1)))
  print("Probability of Detection: ", pod)
  print("Probability of False Detection: ", pofd)
  print("False Alarm Rate: ", far)
  print("Frequency Bias: ", freq_bias)


  # # print(obs_monthly)
  # # print(sat_monthly)

  # # print(obs_monthly_bool)
  # # print(sat_monthly_bool)
  print(" ")

  # obs_monthly = df_obs_monthly[station_id]
  # sat_monthly = df_sat_monthly[station_id]

  # obs_monthly = obs_monthly.to_numpy()
  # sat_monthly = sat_monthly.to_numpy()

  # print(station, station_id)

monthly_metrics_df = monthly_metrics_df.T
monthly_metrics_df.to_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/metrics/metrics_chirps_monthly.csv')      

0
TA00073
Number of Observation:  60
RMSE:  64.2
MAE:  44.2
Bias:  31.8
Rsquare:  0.0
Probability of Detection:  1.0
Probability of False Detection:  nan
False Alarm Rate:  0.21666666666666667
Frequency Bias:  1.2765957446808511
 
1
TA00024
Number of Observation:  60
RMSE:  53.9
MAE:  34.0
Bias:  11.1
Rsquare:  0.7
Probability of Detection:  1.0
Probability of False Detection:  0.0
False Alarm Rate:  0.20689655172413793
Frequency Bias:  1.2608695652173914
 
2
TA00026
Number of Observation:  60
RMSE:  112.6
MAE:  68.6
Bias:  -0.0
Rsquare:  -0.1
Probability of Detection:  1.0
Probability of False Detection:  0.0
False Alarm Rate:  0.1016949152542373
Frequency Bias:  1.1132075471698113
 
3
TA00070
Number of Observation:  60
RMSE:  145.4
MAE:  89.8
Bias:  73.2
Rsquare:  -0.2
Probability of Detection:  1.0
Probability of False Detection:  0.0
False Alarm Rate:  0.3389830508474576
Frequency Bias:  1.5128205128205128
 
4
TA00071
Number of Observation:  60
RMSE:  160.4
MAE:  95.3
Bias:  71.2
R